### Gen AI APP 2

##### Load Data--> Docs-->Divide our Documents into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [38]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://docs.smith.langchain.com/prompt_engineering/concepts')
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/concepts', 'title': 'Concepts | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Prompt engineering is one the core pillars of LangSmith.', 'language': 'en'}, page_content='\n\n\n\n\nConcepts | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringQuickstartsTutorialsOptimize a classifierHow-to GuidesCreate a promptRun the playground against a custom LangServe model serverRun the playground against an OpenAI-compliant model provider/proxyUpdate a promptManage prompts programmaticallyManaging Prompt SettingsPrompt TagsOpen a prompt from a traceLangChain HubPrompt CanvasConceptual GuideDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabil

In [41]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
doc_split = text_splitter.split_documents(docs)
doc_split

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/concepts', 'title': 'Concepts | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Prompt engineering is one the core pillars of LangSmith.', 'language': 'en'}, page_content='Concepts | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/concepts', 'title': 'Concepts | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Prompt engineering is one the core pillars of LangSmith.', 'language': 'en'}, page_content='Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringQuickstartsTutorialsOptimize a classifierHow-to GuidesCreate a promptRun the playground against a custom LangServe model serverRun the playground against an OpenAI-compliant model provider/proxyUpdate a prom

In [1]:
from langchain_ollama import OllamaEmbeddings

embeddings = (
    OllamaEmbeddings(model="gemma:2b")
)


In [43]:
from langchain_community.vectorstores import FAISS

vectorstoredb = FAISS.from_documents(doc_split, embeddings)

vectorstoredb

In [44]:
query="A prompt sets the stage for the model"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'Tags‚Äã\nYou may want to tag prompt commits with a human-readable tag so that you can refer to it even as new commits are added. Common use cases include tagging a prompt with dev or prod tags. This allows you to track which versions of prompts are used where.\nPrompt Playground‚Äã\nThe prompt playground makes the process of iterating and testing your prompts seamless. You can enter the playground from the sidebar or directly from a saved prompt.\nIn the playground you can:\n\nChange the model being used\nChange prompt template being used\nChange the output schema\nChange the tools available\nEnter the input variables to run through the prompt template\nRun the prompt through the model\nObserve the outputs\n\nTesting multiple prompts‚Äã\nYou can add more prompts to your playground to easily compare outputs and decide which version is better:'

In [45]:
from langchain_ollama import ChatOllama

llm=ChatOllama(model='gemma:2b')

In [47]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
        Answer the following question based only on the provided context:
        <context>
        {context}
        </context>

    """
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provided context:\n        <context>\n        {context}\n        </context>\n\n    '), additional_kwargs={})])
| ChatOllama(model='gemma:2b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [54]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Who is the best prompt engineer?",
    "context":[Document(page_content="A prompt sets the stage for the model, like an audience member at an improv show directing the actor's next performance - it guides the model's behavior without changing its underlying capabilities. Just as telling an actor to be a pirate determines how they act, a prompt provides instructions, examples, and context that shape how the model responds. Prompt engineering is important because it allows you to change the way the model behaves. While there are other ways to change the model's behavior (like fine-tuning), prompt engineering is usually the simplest to get started with and often provides the highest ROI. We often see that prompt engineering is multi-disciplinary. Sometimes the best prompt engineer is not the software engineer who is building the application, but rather the product manager or another domain expert. It is important to have the proper tooling and infrastructure to support this cross-disciplinary building.")]
})

'The context does not provide any information about what a prompt is, how it is used, or what its purpose is. Therefore, I cannot answer this question from the provided context.'

In [50]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C80F9898D0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provided context:\n        <context>\n        {context}\n        </context>\n\n    '), addi

In [51]:
response=retrieval_chain.invoke({"input":"What is a prompt in the context of AI models?"})
response['answer']
response

{'input': 'What is a prompt in the context of AI models?',
 'context': [Document(id='6a5ac4ea-375d-453d-aa6c-7e8b8f1b9f12', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/concepts', 'title': 'Concepts | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Prompt engineering is one the core pillars of LangSmith.', 'language': 'en'}, page_content='behavior without changing its underlying capabilities. Just as telling an actor to "be a pirate" determines how they act,\na prompt provides instructions, examples, and context that shape how the model responds.\nPrompt engineering is important because it allows you to change the way the model behaves.\nWhile there are other ways to change the model\'s behavior (like fine-tuning), prompt engineering is usually the simplest to get started with\nand often provides the highest ROI.\nWe often see that prompt engineering is multi-disciplinary.\nSometimes the best prompt engineer is not the software engineer who is building 